# SILVER DATA TRANSFORMATION

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F


In [0]:
df = spark.read.format("delta")\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .load("abfss://bronze@projectdatalakestrgacc.dfs.core.windows.net/netflix_titles")

In [0]:
df = df.fillna({"duration_minutes":0 , "duration_seasons":1})

In [0]:
df_clean = df.filter(
    (F.trim(F.col("duration_minutes")).rlike("^[0-9]+(\\.[0-9]+)?$")) &
    (F.trim(F.col("duration_seasons")).rlike("^[0-9]+(\\.[0-9]+)?$"))
)

df_clean = df_clean.withColumn("duration_minutes", F.col("duration_minutes").cast("int")) \
                   .withColumn("duration_seasons", F.col("duration_seasons").cast("int"))


In [0]:
df = df.withColumn("duration_minutes",col("duration_minutes").cast(IntegerType())).withColumn("duration_seasons",col("duration_seasons").cast(StringType()))

In [0]:
df = df_clean

In [0]:
display(df)

In [0]:
df = df.withColumn("short_titles" , split(col('title'), ':')[0])

In [0]:
df = df.withColumn('rating' , split("rating",'-')[0])

In [0]:
df = df.withColumn('type_flag' , when(col('rating') == 'Movie' , 1)\
                            .when(col('rating') == 'TV' , 2)\
                            .otherwise(0))

In [0]:
display(df)

In [0]:
from pyspark.sql.window import Window

In [0]:
df = df.withColumn("duration_ranking" , dense_rank().over(Window.orderBy(col("duration_minutes").desc())))

In [0]:
display(df)

In [0]:
type_show = df.groupby("type").agg(count("*").alias("total"))
display(type_show)

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path","abfss://silver@projectdatalakestrgacc.dfs.core.windows.net/netflix_titles")\
    .save()